

## 🩺 COVID-19 Detection from CT Scan Images using TensorFlow

### 📌 Project Overview

This project focuses on detecting **COVID-19 from chest CT scan images** using deep learning techniques. A convolutional neural network (CNN) model is developed and trained using **TensorFlow** to classify CT images and assist in automated medical image analysis.

The project is based on the **COVIDx CT dataset** available on Kaggle and aims to demonstrate how deep learning can support medical diagnosis by analyzing CT scan data.

### 📂 Dataset

* **Dataset Source:** COVIDx CT Dataset (Kaggle)  https://www.kaggle.com/datasets/hgunraj/covidxct
* **Data Type:** Chest CT scan images
* **Classes:** COVID-19 positive and non-COVID cases
* The dataset contains labeled CT images collected from real clinical cases.

### 🧠 Model & Approach

* Built using **TensorFlow** and **Keras**
* A **Convolutional Neural Network (CNN)** architecture is used for image classification
* Image preprocessing steps include resizing, normalization, and data augmentation
* The model is trained to learn discriminative features from CT images to identify COVID-19 infections


In [7]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from sklearn.utils import shuffle

In [3]:
DATA_DIR = "data"
IMAGE_DIR = os.path.join(DATA_DIR, "3A_images")

TRAIN_TXT = os.path.join(DATA_DIR, "train_COVIDx_CT-3A.txt")
VAL_TXT   = os.path.join(DATA_DIR, "val_COVIDx_CT-3A.txt")
TEST_TXT  = os.path.join(DATA_DIR, "test_COVIDx_CT-3A.txt")


In [4]:
def load_txt_to_df(txt_path):
    filenames = []
    labels = []

    with open(txt_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 6:
                continue

            fname, cls, _, _, _, _ = parts[:]
            cls = int(cls)

            label = 1 if cls == 2 else 0

            filenames.append(fname)
            labels.append(label)

    df = pd.DataFrame({
        "filename": filenames,
        "label": labels
    })

    df["path"] = df["filename"].apply(lambda x: os.path.join(IMAGE_DIR, x))
    return df

In [5]:
train_df = load_txt_to_df(TRAIN_TXT)
val_df   = load_txt_to_df(VAL_TXT)
test_df  = load_txt_to_df(TEST_TXT)

In [8]:
all_df = pd.concat([train_df, val_df, test_df], ignore_index=True)
all_df = shuffle(all_df, random_state=1988).reset_index(drop=True)

In [9]:
all_df.shape

(425024, 3)

### For running the model on the system here, I have limited the number of samples in the train/test split. By using a GPU, the full dataset can be used.

In [12]:
train_df = all_df.iloc[:8000].reset_index(drop=True)
val_df   = all_df.iloc[8000:9000].reset_index(drop=True)
test_df  = all_df.iloc[9000:10000].reset_index(drop=True)

In [13]:
len(train_df), len(val_df), len(test_df)


(8000, 1000, 1000)

In [14]:
IMG_SIZE = 224
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

In [15]:
def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

In [17]:
def make_dataset(df, shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices(
        (df["path"].values, df["label"].values)
    )
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds


In [18]:
train_ds = make_dataset(train_df, shuffle=True)
val_ds   = make_dataset(val_df)
test_ds  = make_dataset(test_df)


In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, CSVLogger
from tensorflow.keras.optimizers import Adam
import datetime


model = Sequential([
    Conv2D(32, (3,3), activation="relu", padding="same", input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    BatchNormalization(),
    MaxPool2D(2),

    Conv2D(64, (3,3), activation="relu", padding="same"),
    BatchNormalization(),
    MaxPool2D(2),

    Conv2D(128, (3,3), activation="relu", padding="same"),
    BatchNormalization(),
    MaxPool2D(2),

    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

optimizer = Adam(learning_rate=1e-4)  # stable learning rate
model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = TensorBoard(log_dir=log_dir, histogram_freq=1)

csv_logger = CSVLogger("training_log.csv", append=True)

callbacks = [early_stop, tensorboard_cb, csv_logger]


history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=callbacks
)


test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


Epoch 1/50


/Users/mohammadshafiei/Desktop/project/.pixi/envs/default/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 191s 740ms/step - accuracy: 0.8462 - loss: 0.4834 - val_accuracy: 0.7260 - val_loss: 3.2979
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 184s 719ms/step - accuracy: 0.9170 - loss: 0.2184 - val_accuracy: 0.7720 - val_loss: 0.8618
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 182s 712ms/step - accuracy: 0.9358 - loss: 0.1623 - val_accuracy: 0.9380 - val_loss: 0.1639
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 181s 711ms/step - accuracy: 0.9501 - loss: 0.1245 - val_accuracy: 0.9330 - val_loss: 0.1588
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 184s 719ms/step - accuracy: 0.9563 - loss: 0.1045 - val_accuracy: 0.9540 - val_loss: 0.1368
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 191s 752ms/step - accuracy: 0.9649 - loss: 0.0849 - val_accuracy: 0.9390 - val_loss: 0.1584
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 238s 939ms/step - accuracy: 0.9716 - loss: 0.0669 - val_accuracy: 0.9590 - val_loss: 0.1173
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 256s 1s/step - accuracy: 0.9743 - loss: 0.0584 

KeyboardInterrupt: 